In [1]:
# python setup 
import pandas as pd 
import time
import numpy as np 
from datetime import datetime, timezone, timedelta

# Introduction 
In this notebool, we are going to develop some analytic tool to extract summary information from the tutor log data. There will also be code chunks distilling event-actor-subject data from the tutor log. 

The analytics tools will include: 
- Summary of student performance, i.e., percentage of correct attempts, etc.. 
- Average number of hints 
- Time taken to solve (mean, std) 
- Total number of problems solved 
- Total KC level performance 
- Struggle summary stats with data from LearnSphere Critical Struggle detector 

# Summary Functions 
Below are functions that generates summary statistics mentioned above. 

In [2]:
def EDTDatetime2epoch(dateTime, format="%Y-%m-%d %H:%M:%S"):
    """Converts EDT date-time string to epoch time represented by an interger 

    Args:
        datetime (str): date-time string, in EDT time zone
        format (str, optional): python time module time string formats, read more in `time` documentation. Defaults to "%Y-%m-%d %H:%M:%S".

    Returns: 
        int: epoch/unix time stamp integer
    """    

    # read-in datetime string
    datetimeStruct = datetime.strptime(dateTime, format) 
    # add time zone informartion 
    datetimeStruct = datetimeStruct.replace(tzinfo=timezone(timedelta(hours=-4))) 
    timestamp = datetimeStruct.timestamp()
    return timestamp 

def UTCDatetime2epoch(dateTime, format="%Y-%m-%d %H:%M:%S"):
    """Converts UTC date-time string to epoch time represented by an interger 

    Args:
        datetime (str): date-time string, in UTC time zone
        format (str, optional): python time module time string formats, read more in `time` documentation. Defaults to "%Y-%m-%d %H:%M:%S".

    Returns: 
        int: epoch/unix time stamp integer
    """    

    # read-in datetime string
    datetimeStruct = datetime.strptime(dateTime, format) 
    # add time zone informartion 
    datetimeStruct = datetimeStruct.replace(tzinfo=timezone.utc) 
    timestamp = datetimeStruct.timestamp()
    return timestamp 

def epoch2datetimeInEDT(timestamp):
    """Converts epoch time stamp to date-time string in EDT time zone 

    Args:
        timestamp (int or float): epoch time stamp 

    Returns:
        string: date-time string in EDT time zone
    """    

    assert(timestamp >= 0)

    dateTime = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
    return dateTime 

def filterWithStudents(tutorLogDF, students): 
    """filter tutor log dataframe by students' anon user ids and return a filtered dataset 

    Args:
        tutorLogDF (pd.DataFrame): tutor log data set 
        students (None or Iterable): an iterable with desired student anon ids 

    Returns:
        pd.DataFrame: a filtered dataset with just these wanted students 
    """    
    assert(students == None or len(students) > 0) # input check 

    filteredDF = tutorLogDF.copy()
    if(students != None): filteredDF = tutorLogDF.loc[tutorLogDF["Anon Student Id"].isin(students)] 

    return filteredDF

def filterWithTime(tutorLogDF, startTime, endTime):
    """filter tutor log data by start and end time stamp. Usually used to extract data from a period 

    Args:
        tutorLogDF (pd.DataFrame): tutor log data set 
        startTime (int): start time stamp
        endTime (int): end time stamp

    Returns:
        pd.DataFrame: filtered dataset with entries from the time stamp interval 
    """    
    
    assert(startTime == None or endTime == None or endTime > startTime) # input check 

    # filteredDF = tutorLogDF.copy()
    # if(startTime != None): filteredDF = filteredDF.loc[filteredDF["timestamp"] >= startTime]
    # if(endTime != None): filteredDF = filteredDF.loc[filteredDF["timestamp"] <= endTime] 
    
    filteredDF = tutorLogDF.copy()
    if(startTime != None): filteredDF = tutorLogDF.loc[tutorLogDF["timestamp"] >= startTime]
    if(endTime != None): filteredDF = filteredDF.loc[filteredDF["timestamp"] <= endTime] 

    return filteredDF

def getStudentPerformanceSummary(tutorLogDF, students=None, startTime=None, endTime=None):
    """
    Measures student performance by percentage of attempt correctness during a time interval

    Args:
        tutorLogDF (pd.Dataframe): tutor log dataframe imported from a datashop exported file 
        students (Iterable): one/several anon_stud_id's to get the performance percentage 
        startTime (_int_): unix time stamp indicating the start of the interval 
        endTime (_int_): unix time stamp indicating the end of the interval 

    Returns: 
        float: a float representing attempt correct rate
        
    """ 

    # input check 
    assert(students == None or len(students) > 0) 
    assert(startTime == None or endTime == None or endTime > startTime) 

    # only get rows with desired students and start/end time 
    filteredDF = tutorLogDF.copy()
    if(students != None): filteredDF = tutorLogDF.loc[tutorLogDF["Anon Student Id"].isin(students)] 
    if(startTime != None): filteredDF = filteredDF.loc[filteredDF["timestamp"] >= startTime]
    if(endTime != None): filteredDF = filteredDF.loc[filteredDF["timestamp"] <= endTime] 

    # total number of attempt = number of correct attempt + number of incorrect + number of hints requested 
    totalCorrect = filteredDF["Outcome"].value_counts().get("CORRECT", 0)
    totalIncorrect = filteredDF["Outcome"].value_counts().get("INCORRECT", 0)
    totalHint = filteredDF["Outcome"].value_counts().get("HINT", 0)
    totalAttempts = totalCorrect + totalIncorrect + totalHint

    # Tried some validation. However, the number of ATTEMPS values in the 
    # `Student Response Type` column does not match with CORRECT and INCORRECT
    # outcomes, so the total number of attempts will be the sum of the two

    # if(totalAttempts == 0): return 0
    if(totalAttempts == 0): return np.nan
    else: return totalCorrect / totalAttempts


def getNumOfProblemsSolved(tutorLogDF, students=None, startTime=None, endTime=None): 
    """Generate descriptive stats of the total number of problems solved by given students during given time interval 

    Args:
        tutorLogDF (pd.Dataframe): tutor log dataframe imported from a datashop exported file 
        students (Iterable): one/several anon_stud_id's to get the performance percentage 
        startTime (_int_): unix time stamp indicating the start of the interval 
        endTime (_int_): unix time stamp indicating the end of the interval 

    Returns:
        int: total number of problems solved by given students during given time interval 
    """    

    # do some filtering with students and start/end timestamp 
    filteredDF = filterWithStudents(tutorLogDF, students) 
    filteredDF = filterWithTime(filteredDF, startTime, endTime) 

    # Each problem should correspond to one and only one `done ButtonPressed`, 
    # so we can use the "done ButtonPressed" values in the `Step Name` column to 
    # extract the number of problems solved with these students between this 
    # time interval
    numOfProblemsSolved = filteredDF["Step Name"].value_counts().get("done ButtonPressed", 0)

    return numOfProblemsSolved

def getNumOfHints(tutorLogDF, students=None, startTime=None, endTime=None): 
    """Generate descriptive stats of the total number of hints requested by given students during given time interval 

    Args:
        tutorLogDF (pd.Dataframe): tutor log dataframe imported from a datashop exported file 
        students (Iterable): one/several anon_stud_id's to get the performance percentage 
        startTime (_int_): unix time stamp indicating the start of the interval 
        endTime (_int_): unix time stamp indicating the end of the interval 

    Returns:
        int: total number of hints requested by given students during given time interval 
    """  

    # do some filtering with students and start/end timestamp 
    filteredDF = filterWithStudents(tutorLogDF, students) 
    filteredDF = filterWithTime(filteredDF, startTime, endTime) 

    numOfHints = filteredDF["Student Response Type"].value_counts().get("HINT_REQUEST", 0)

    return numOfHints


def getAveNumOfHintsPerProblem(tutorLogDF, students=None, startTime=None, endTime=None): 
    """Generates average number of hints used by given students in given time period 

    Args:
        tutorLogDF (pd.Dataframe): tutor log dataframe imported from a datashop exported file 
        students (Iterable): one/several anon_stud_id's to get the performance percentage 
        startTime (_int_): unix time stamp indicating the start of the interval 
        endTime (_int_): unix time stamp indicating the end of the interval 

    Returns:
        float: average number of hints used by given students in given time period
    """  

    numOfHints = getNumOfHints(tutorLogDF, students=students, startTime=startTime, endTime=endTime) 
    numOfProblems = getNumOfProblemsSolved(tutorLogDF, students=students, startTime=startTime, endTime=endTime) 
    assert numOfProblems != 0, "there is no problem solved with given conditions" 

    return numOfHints / numOfProblems 

def getTimeToSolveSummary(tutorLogDF, students=None, startTime=None, endTime=None): 
    """ 
    Get the mean and std the time, in seconds, to solve each problem for 
    given student in given time interval

    Args:
        tutorLogDF (pd.Dataframe): tutor log dataframe imported from a datashop exported file 
        students (Iterable): one/several anon_stud_id's to get the performance percentage 
        startTime (_int_): unix time stamp indicating the start of the interval 
        endTime (_int_): unix time stamp indicating the end of the interval 

    Returns:
        (float, float): mean and std of time taken to solve the problems (unit is second)
    """    

    timeTakenForOneProblemUpperBound = 60*20 # in second 

    # basic filtering 
    filteredDF = filterWithStudents(tutorLogDF, students) 
    filteredDF = filterWithTime(filteredDF, startTime, endTime) 

    # rows with "done ButtonPressed" as value of `Step Name` should indicate the 
    # last transaction of the problem 
    filteredDF = filteredDF.loc[filteredDF["Step Name"] == "done ButtonPressed"] 
    # the difference in time between the "done ButtonPressed" transaction and 
    # the `Problem Start Time` value should be time taken to solve the problem
    timeTaken = filteredDF["Time"].apply(UTCDatetime2epoch) - filteredDF["Problem Start Time"].apply(UTCDatetime2epoch) 
    timeTaken = timeTaken.loc[timeTaken < timeTakenForOneProblemUpperBound]

    return np.mean(timeTaken), np.std(timeTaken) 

def getKCLevelPerformance(tutorLogDF, students=None, startTime=None, endTime=None): 
    """Generate the performance of given students in given time interval in each KC levels 

    Args:
        tutorLogDF (pd.Dataframe): tutor log dataframe imported from a datashop exported file 
        students (Iterable): one/several anon_stud_id's to get the performance percentage 
        startTime (_int_): unix time stamp indicating the start of the interval 
        endTime (_int_): unix time stamp indicating the end of the interval 

    Returns:
        dict: a dictionary mapping from KC's (string) to its correct rate (float)
    """    

    # basic filtering 
    filteredDF = filterWithStudents(tutorLogDF, students) 
    filteredDF = filterWithTime(filteredDF, startTime, endTime) 

    # all KC levels are here: 
    KCLevels = ['cancel-const', 'division-simple', 'divide',
                'subtraction-const', 'combine-like-const', 'subtraction-var',
                'combine-like-var', 'cancel-var', 'distribute-division',
                'division-complex'] 
    kc2CorrectRateMapping = dict() # to be returned 

    for kc in KCLevels: 
        kcDF = filteredDF.loc[filteredDF["KC (Default)"] == kc] # dataframe with this KC only 
        correctRate = getStudentPerformanceSummary(kcDF) # calculate correct rate with this KC
        kc2CorrectRateMapping[kc + "_rate"] = correctRate # populate the mapping dictionary
    
    return kc2CorrectRateMapping


def getProblemLevelSummary(tutorLogDF, students=None, startTime=None, endTime=None): 

    # basic filtering 
    filteredDF = filterWithStudents(tutorLogDF, students) 
    filteredDF = filterWithTime(filteredDF, startTime, endTime) 

    # get the last transaction for each problem since it is unique 
    filteredDF = filteredDF.loc[filteredDF["Step Name"] == "done ButtonPressed"] 

    # return problem level mean and std 
    return np.mean(filteredDF["Level (Position)"]), np.std(filteredDF["Level (Position)"])


def getNumOfSteps(tutorLogDF, students=None, startTime=None, endTime=None):

    # basic filtering 
    filteredDF = filterWithStudents(tutorLogDF, students) 
    filteredDF = filterWithTime(filteredDF, startTime, endTime) 
    # total number of not-NaN values in `Step Name` column is the number of steps
    numOfSteps = filteredDF["Step Name"].notnull().sum()

    return numOfSteps


In [3]:
# data-loading 
if __name__ =="__main__": 
    tutorLogDF = pd.read_csv("raw data/tutor_log.tsv", delimiter="\t", index_col=False) 
    tutorLogDF["Time Zone"] = "UTC" # the logs are entered in UTC time zone 
    tutorLogDF["timestamp"] = tutorLogDF["Time"].apply(UTCDatetime2epoch) # add a new column with unix time stamps 
    tutorLogDF["EDT_time"] = tutorLogDF["timestamp"].apply(epoch2datetimeInEDT) # append a new column with EDT time information to be more intuitive 

    # only aceepting data within the experiment period, which is between 05/23/2022 and 05/25/2022 
    experimentStartTimestamp = EDTDatetime2epoch("2022-05-23 08:00:00")
    experimentEndTimestamp = EDTDatetime2epoch("2022-05-25 16:00:00")
    tutorLogDF = filterWithTime(tutorLogDF, experimentStartTimestamp, experimentEndTimestamp) 
    # tutorLogDF.hist(column="timestamp", bins=50) # validate by graph  

def getAnnotatedTutorLogDF():
    tutorLogDF = pd.read_csv("raw data/tutor_log.tsv", delimiter="\t", index_col=False) 
    tutorLogDF["Time Zone"] = "UTC" # the logs are entered in UTC time zone 
    tutorLogDF["timestamp"] = tutorLogDF["Time"].apply(UTCDatetime2epoch) # add a new column with unix time stamps 
    tutorLogDF["EDT_time"] = tutorLogDF["timestamp"].apply(epoch2datetimeInEDT) # append a new column with EDT time information to be more intuitive 

    # only aceepting data within the experiment period, which is between 05/23/2022 and 05/25/2022 
    experimentStartTimestamp = EDTDatetime2epoch("2022-05-23 08:00:00")
    experimentEndTimestamp = EDTDatetime2epoch("2022-05-25 16:00:00")
    tutorLogDF = filterWithTime(tutorLogDF, experimentStartTimestamp, experimentEndTimestamp) 

    return tutorLogDF


/var/folders/jk/t2z3d3zd447g4wlv0rvml1400000gn/T/ipykernel_56615/1388600548.py:3: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  tutorLogDF = pd.read_csv("raw data/tutor_log.tsv", delimiter="\t", index_col=False)


In [4]:
# data summary chunk 
if __name__ == "__main__": 
    print("Below are some besic specs:")
    print("Correct and incorrect attempt proportions are", getStudentPerformanceSummary(tutorLogDF))
    print("Total number of problems solved in the three days:", getNumOfProblemsSolved(tutorLogDF)) 
    print("Total number of hints requested in the three days", getNumOfHints(tutorLogDF)) 
    print("Average number of hints per question for all students in the three days:", getAveNumOfHintsPerProblem(tutorLogDF))
    print("Mean and STD of time taken to solve the problems are:", getTimeToSolveSummary(tutorLogDF)) 
    print("KC level performance summary:")
    print(getKCLevelPerformance(tutorLogDF))
    print("Problem level summary:", getProblemLevelSummary(tutorLogDF))



Below are some besic specs:
Correct and incorrect attempt proportions are 0.8012281100750511
Total number of problems solved in the three days: 2799
Total number of hints requested in the three days 4043
Average number of hints per question for all students in the three days: 1.4444444444444444
Mean and STD of time taken to solve the problems are: (92.36132952108649, 100.60584470353174)
KC level performance summary:
{'cancel-const_rate': 0.9585714285714285, 'division-simple_rate': 0.5222551928783383, 'divide_rate': 0.9910198845413727, 'subtraction-const_rate': 0.3211091234347048, 'combine-like-const_rate': 0.7168949771689498, 'subtraction-var_rate': 0.37844611528822053, 'combine-like-var_rate': 0.6990291262135923, 'cancel-var_rate': 0.9671412924424972, 'distribute-division_rate': 0.23809523809523808, 'division-complex_rate': 0.3684210526315789}
Problem level summary: (4.159699892818864, 2.9110680701431177)


In [5]:
# summary by days and periods 

# start and end time of each day/period 
               # period1                                  period2                                   period3                                   period4                                   period5
startTimes =  [ 
              [EDTDatetime2epoch("2022-05-23 08:26:00"), EDTDatetime2epoch("2022-05-23 10:13:00"), EDTDatetime2epoch("2022-05-23 11:05:00"), EDTDatetime2epoch("2022-05-23 12:36:00"), EDTDatetime2epoch("2022-05-23 14:17:00")], # day1
              [EDTDatetime2epoch("2022-05-24 08:21:00"), EDTDatetime2epoch("2022-05-24 10:02:00"), EDTDatetime2epoch("2022-05-24 10:57:00"), EDTDatetime2epoch("2022-05-24 12:24:00"), EDTDatetime2epoch("2022-05-24 14:07:00")], # day2
              [EDTDatetime2epoch("2022-05-25 08:21:00"), EDTDatetime2epoch("2022-05-25 10:01:00"), EDTDatetime2epoch("2022-05-25 10:55:00"), EDTDatetime2epoch("2022-05-25 12:24:00"), EDTDatetime2epoch("2022-05-25 14:07:00")]  # day3
              ] 

               # period1                                  period2                                   period3                                   period4                                   period5
endTimes =  [ 
            [EDTDatetime2epoch("2022-05-23 08:53:00"), EDTDatetime2epoch("2022-05-23 10:41:00"), EDTDatetime2epoch("2022-05-23 11:30:00"), EDTDatetime2epoch("2022-05-23 13:00:00"), EDTDatetime2epoch("2022-05-23 14:40:00")], # day1
            [EDTDatetime2epoch("2022-05-24 08:43:00"), EDTDatetime2epoch("2022-05-24 10:27:00"), EDTDatetime2epoch("2022-05-24 11:15:00"), EDTDatetime2epoch("2022-05-24 12:48:00"), EDTDatetime2epoch("2022-05-24 14:30:00")], # day2
            [EDTDatetime2epoch("2022-05-25 08:43:00"), EDTDatetime2epoch("2022-05-25 10:24:00"), EDTDatetime2epoch("2022-05-25 11:18:00"), EDTDatetime2epoch("2022-05-25 12:47:00"), EDTDatetime2epoch("2022-05-25 14:30:00")]  # day3
            ] 

def getPeriodsStartEndTimes(): 
    return startTimes, endTimes 

summaryDF = pd.DataFrame(columns=["dayID", "periodID", "correct_attempt_rate", "total_n_problems", "total_n_hints", 
                                  "ave_n_hint", "time_per_problem_mean", "time_per_problem_sd", 
                                  "problem_level_mean", "problem_level_std",
                                  'cancel-const_rate', 'division-simple_rate', 'divide_rate', 
                                  'subtraction-const_rate', 'combine-like-const_rate', 
                                  'subtraction-var_rate', 'combine-like-var_rate', 'cancel-var_rate', 
                                  'distribute-division_rate', 'division-complex_rate'])

# column names of the KC level performance 
kc_rate_strings = ['cancel-const_rate', 'division-simple_rate', 'divide_rate', 
                   'subtraction-const_rate', 'combine-like-const_rate', 
                   'subtraction-var_rate', 'combine-like-var_rate', 'cancel-var_rate', 
                   'distribute-division_rate', 'division-complex_rate']


if __name__ == "__main__": 

    days = 3
    periods = 5
    for day in range(days): 
        for period in range(periods): 
            # start and end time of the given day/period 
            startTime = startTimes[day][period] 
            endTime = endTimes[day][period] 
            filteredDF = filterWithTime(tutorLogDF, startTime, endTime) # get data in given time interval 

            # create a new row with populated values 
            newRowDict = {"correct_attempt_rate": [getStudentPerformanceSummary(filteredDF)], 
                          "total_n_problems": [getNumOfProblemsSolved(filteredDF)], 
                          "total_n_hints": [getNumOfHints(filteredDF)], 
                          "ave_n_hint": [getAveNumOfHintsPerProblem(filteredDF)], 
                          "time_per_problem_mean": [getTimeToSolveSummary(filteredDF)[0]], 
                          "time_per_problem_sd": [getTimeToSolveSummary(filteredDF)[1]],
                          "problem_level_mean": [getProblemLevelSummary(filteredDF)[0]], 
                          "problem_level_std": [getProblemLevelSummary(filteredDF)[1]],
                          "dayID": [day+1], 
                          "periodID": [period+1]
                         } 

            kc_results = getKCLevelPerformance(filteredDF) # get performance stats in KC level, this is a dictionary 
            # populate the KC level performance columns 
            for kc_rate_string in kc_rate_strings: 
                newRowDict[kc_rate_string] = [kc_results[kc_rate_string]]

            newDF = pd.DataFrame(newRowDict) 
            summaryDF = pd.concat([summaryDF, newDF], ignore_index=True)


In [6]:
# write the summary data file 
if __name__ == "__main": 
    summaryDF.to_csv("output_files/tutor_summary.csv", index=False)